In [1]:
# import pandas as pd
from methods.make_single_df_from_bid_ask import make_single_df_from_bid_ask
# from methods.make_5min_volume_from_1_min_volume import make_5min_volume_from_1min_volume
# from machine_learning_models.lstm.eurusd.lstm_model_stats import predict_max_stats, predict_min_stats
# import torch

In [2]:
BASE_DIR = 'hist_data/'
TIME_SERIES_FOLDER = 'EURUSD/'
BID_FOLDER = '5_min/Bid/'
ASK_FOLDER = '5_min/Ask/'

BASE_DIR_LSTM = 'machine_learning_models/lstm/eurusd/pickle_files'
LSTM_MAX_VAL = BASE_DIR_LSTM + '/lstm_regressor_predict_max_values-2025-04-07.pkl'
LSTM_MIN_VAL = BASE_DIR_LSTM + '/lstm_regressor_predict_min_values-2025-04-07.pkl'
SCALER_X_MIN = BASE_DIR_LSTM + '/lstm_regressor_scaler_x_min.pkl'
SCALER_X_MAX = BASE_DIR_LSTM + '/lstm_regressor_scaler_x_max.pkl'
SCALER_Y_MIN = BASE_DIR_LSTM + '/lstm_regressor_scaler_y_min.pkl'
SCALER_Y_MAX = BASE_DIR_LSTM + '/lstm_regressor_scaler_y_max.pkl'

In [3]:
df_joined_train, df_joined_test, df_joined_val = make_single_df_from_bid_ask(
    base_dir=BASE_DIR,
    time_series_folder=TIME_SERIES_FOLDER
)

d:\PYTHON_PROJECTS\python_algo_ai\methods\return_single_large_dataframe.py:15: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  final_df = pd.concat([final_df, new_df])
d:\PYTHON_PROJECTS\python_algo_ai\methods\return_single_large_dataframe.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat([final_df, new_df])
d:\PYTHON_PROJECTS\python_algo_ai\methods\return_single_large_dataframe.py:15: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no 

In [4]:
df_joined_train

,open,close,high,low,volume,+DI,-DI,ADX,open_audusd,close_audusd,...,low_plus_30min,close_plus_30min,open_plus_35min,high_plus_35min,low_plus_35min,close_plus_35min,open_plus_40min,high_plus_40min,low_plus_40min,close_plus_40min
2022-01-02 22:00:00,1.13776,1.13689,1.13776,1.13689,8.240,0.000000,0.000000,NaN,NaN,NaN,...,1.13700,1.13766,1.13765,1.13776,1.13722,1.13728,1.13729,1.13774,1.13729,1.13773
2022-01-02 22:05:00,1.13776,1.13689,1.13776,1.13649,121.085,0.000000,18.691589,100.000000,NaN,NaN,...,1.13722,1.13728,1.13729,1.13774,1.13729,1.13773,1.13773,1.13773,1.13730,1.13730
2022-01-02 22:10:00,1.13682,1.13774,1.13777,1.13678,222.920,0.319489,12.779553,97.560976,0.72538,0.72737,...,1.13729,1.13773,1.13773,1.13773,1.13730,1.13730,1.13759,1.13766,1.13729,1.13730
2022-01-02 22:15:00,1.13772,1.13712,1.13772,1.13708,41.760,-1.055409,10.554090,105.781391,0.72736,0.72550,...,1.13730,1.13730,1.13759,1.13766,1.13729,1.13730,1.13756,1.13766,1.13727,1.13729
2022-01-02 22:20:00,1.13752,1.13702,1.13758,1.13700,272.820,-0.915332,10.983982,108.881498,0.72741,0.72550,...,1.13729,1.13730,1.13756,1.13766,1.13727,1.13729,1.13721,1.13802,1.13721,1.13786
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-01-04 10:35:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.67400,0.67388,...,1.09535,1.09539,1.09538,1.09548,1.09533,1.09542,1.09543,1.09553,1.09508,1.09527
2024-01-04 10:40:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.67400,0.67344,...,1.09533,1.09542,1.09543,1.09553,1.09508,1.09527,1.09528,1.09542,1.09498,1.09518
2024-01-04 10:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.67352,0.67328,...,1.09508,1.09527,1.09528,1.09542,1.09498,1.09518,1.09518,1.09565,1.09516,1.09550
2024-01-04 10:50:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.67339,0.67320,...,1.09498,1.09518,1.09518,1.09565,1.09516,1.09550,1.09550,1.09580,1.09550,1.09560


In [5]:
df_joined_test.tail()

,open,close,high,low,volume,+DI,-DI,ADX,open_audusd,close_audusd,...,low_plus_30min,close_plus_30min,open_plus_35min,high_plus_35min,low_plus_35min,close_plus_35min,open_plus_40min,high_plus_40min,low_plus_40min,close_plus_40min
2024-09-04 09:15:00,1.10533,1.10531,1.10539,1.10509,1390.85,20.466321,17.616580,12.522040,0.67062,0.67092,...,1.10554,1.10559,1.10560,1.10565,1.10539,1.10545,1.10548,1.10548,1.10529,1.10531
2024-09-04 09:20:00,1.10532,1.10550,1.10556,1.10526,1991.55,20.365535,17.754569,11.406882,0.67083,0.67099,...,1.10539,1.10545,1.10548,1.10548,1.10529,1.10531,1.10534,1.10535,1.10510,1.10517
2024-09-04 09:25:00,1.10548,1.10537,1.10558,1.10536,1271.19,19.841270,17.989418,9.830680,0.67101,0.67089,...,1.10529,1.10531,1.10534,1.10535,1.10510,1.10517,1.10517,1.10525,1.10511,1.10521
2024-09-04 09:30:00,1.10536,1.10555,1.10561,1.10528,1306.35,9.562842,20.765027,12.307384,0.67088,0.67132,...,1.10510,1.10517,1.10517,1.10525,1.10511,1.10521,1.10519,1.10535,1.10507,1.10530
2024-09-04 09:35:00,1.10554,1.10566,1.10582,1.10554,933.38,15.819209,19.774011,12.631935,0.67122,0.67153,...,1.10511,1.10521,1.10519,1.10535,1.10507,1.10530,1.10525,1.10537,1.10516,1.10537


In [6]:
df_joined_val.head()

,open,close,high,low,volume,+DI,-DI,ADX,open_audusd,close_audusd,...,low_plus_30min,close_plus_30min,open_plus_35min,high_plus_35min,low_plus_35min,close_plus_35min,open_plus_40min,high_plus_40min,low_plus_40min,close_plus_40min
2024-09-04 09:40:00,1.10563,1.10571,1.10586,1.10559,1020.65,15.804598,20.114943,12.003583,0.67152,0.67143,...,1.10507,1.10530,1.10525,1.10537,1.10516,1.10537,1.10536,1.10540,1.10514,1.10534
2024-09-04 09:45:00,1.10570,1.10559,1.10572,1.10554,942.21,16.369048,22.321429,12.577573,0.67142,0.67164,...,1.10516,1.10537,1.10536,1.10540,1.10514,1.10534,1.10536,1.10543,1.10527,1.10529
2024-09-04 09:50:00,1.10560,1.10545,1.10565,1.10539,1369.61,16.369048,21.726190,13.041387,0.67164,0.67139,...,1.10514,1.10534,1.10536,1.10543,1.10527,1.10529,1.10527,1.10552,1.10524,1.10549
2024-09-04 09:55:00,1.10548,1.10531,1.10548,1.10529,2165.65,17.684887,18.649518,11.792038,0.67147,0.67137,...,1.10527,1.10529,1.10527,1.10552,1.10524,1.10549,1.10546,1.10555,1.10541,1.10554
2024-09-04 10:00:00,1.10534,1.10517,1.10535,1.10510,2779.10,17.857143,25.000000,12.049239,0.67147,0.67125,...,1.10524,1.10549,1.10546,1.10555,1.10541,1.10554,1.10554,1.10559,1.10521,1.10527


In [8]:
print(len(df_joined_train), len(df_joined_test), len(df_joined_val))

273243 91081 91082


In [5]:
predict_min_stats(df_joined_train, df_joined_test, df_joined_val, scaler_x_dir=SCALER_X_MIN, scaler_y_dir=SCALER_Y_MIN, model_dir=LSTM_MIN_VAL)

Deleting raw dataframes
Raw dataframes deleted
*******************************
After training on test sample:
*******************************
Epoch 10/10 | Train Loss: 0.004747 | Test Loss: 0.005189 | Validation loss: 0.005559

Final Train RMSE: 0.01 | Test RMSE: 0.01 | Validation RMSE: 0.01


In [ ]:
predict_max_stats(df_joined_train, df_joined_test, df_joined_val, scaler_x_dir=SCALER_X_MAX, scaler_y_dir=SCALER_Y_MAX, model_dir=LSTM_MAX_VAL)